# Desarrollo de un script para cálculo de circuitos magnéticos

Leyes fundamentales:



https://www.researchgate.net/publication/229010144_Modeling_of_Nonlinear_Ferromagnetic_Cores

https://www.researchgate.net/publication/239397876_A_phenomenological_mathematical_model_of_hysteresis 


## Instrucciones de ejecución:

-Presione Ctrl+F9 para ejecutar el script completo.

-Busque los delizadores interactivos para ajustar los grados de libertad y las variables de excitación de los núcleos.


-Visualize los resultados y gráficas ofrecidos por cada celda interactiva.




In [ ]:
#LIBRERIAS IMPORTADAS PARA EL PROYECTO:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact

#DEFINICION DE CONSTANTES BASICAS:
mu_0= 1.25663706212*10**(-6)        #PERMEABILIDAD DEL VACÍO 
e_0=8.85418782 * 10**(-12)        #PERMITIVIDAD DEL VACÍO
v=1/np.sqrt(mu_0*e_0)             #VELOCIDAD DE LA LUZ

#FUNCIONES BÁSICAS:
def paralelo(a,b):
  par=a*b/(a+b)     #PARALELO DE RELUCTANCIA
  return par

def sech(x):
  return 1/np.cosh(x)   #SECANTE HIPERBÓLICA

"""

/************************************CLASES "ESQUELETO" DE LOS NÚCLEOS***************************************/
**
**
**
*****************SE ESTRUCTURA LA PLANTILLA DE ATRIBUTOS QUE TENDRÁN LOS NÚCLEOS***************************

"""


class NucleoSimple:        
  def __init__(self,area,largo,mu_r ,N, lineal=True,a=0.85):
    """
    Parámetros de entrada para el núcleo ferromagnético:

    area: área del núcleo en metros cuadrados

    largo: largo efectivo del nucleo en metros

    mu_r: permeabilidad relativa del material del núcleo

    N: número de vueltas del devanado excitador del núcleo

    lineal: booleano indicador de si el nucleo es lineal o no, por defecto es verdadero

    a: constante de fuerza coercitiva   

    """
    self.area=area
    self.largo=largo
    self.perm=mu_r*mu_0               #PERMEABILIDAD RELATIVA POR PERMEABILIDAD DEL VACÍO
    self.vueltas=N
    self.hay_reluctancia=False                #AUN NO SE HA HALLADO LA RELUCTANCIA 
    self.lineal=lineal    
    self.a=a


  def plot(self):
    H=np.linspace(-10000,10000,100000)        #ESPACIO EN EL QUE VIVE H
    if (self.lineal):
      B=self.perm*H
    else:      
      B0=2
      C0=0.00054
      A0=0.000014
      C3=-0.05
      B=B0*np.tanh(C0*H)*(1-2*C3*sech(C0*H)**2 )+A0*H     #SE CALCULA LA CURVA B


    plt.figure(figsize=(16,6))
    plt.subplot(121)
    #CURVA DE SATURACIÓN
    plt.plot(H,B)
    plt.xlim(0,10000)
    plt.ylim(0,max(B)*1.2)
    plt.grid()
    plt.title("Curva B-H")
    plt.xlabel('H [A vueltas/m] ')
    plt.ylabel("B [T] ")
    
    #-----------------------------------
    
    #CURVA DE HISTÉRESIS
    plt.subplot(122)
    plt.plot(H,B)
    if (not self.lineal):
      a=self.a
      xm=1000
      b=0.5*(np.tanh(xm+a)-np.tanh(xm-a))    
      B_plus=B0*np.tanh(C0*H+a)-b+A0*H    #CURVA SUBIDA
      B_minus=B0*np.tanh(C0*H-a)+b+A0*H   #CURVA DE BAJADA
      plt.plot(H,B_plus)
      plt.plot(H,B_minus)

    plt.grid()
    plt.title("Curva de histéresis")
    plt.xlabel('H [A vuelta/m] ')
    plt.ylabel("B [T] ")

 
  def calcular_reluctancia(self):
    self.reluctancia=self.largo/(self.perm*self.area)
    self.hay_reluctancia=True
    return(self.reluctancia)
  def calcular_flujo(self,corriente):
    if (self.hay_reluctancia):      
      self.flujo=corriente*self.vueltas/self.reluctancia
    else:
      self.reluctancia=self.calcular_reluctancia()
      self.flujo=corriente*self.vueltas/self.reluctancia
    return self.flujo



class Nucleo:        
  def __init__(self,l,h,w1,w2,w3,w4,d,mu_r ,N, lineal=True):
    """
    Parámetros de entrada para el núcleo ferromagnético:

    area: área del núcleo en metros cuadrados

    largo: largo efectivo del nucleo en metros

    mu_r: permeabilidad relativa del material del núcleo

    N: número de vueltas del devanado excitador del núcleo

    lineal: booleano indicador de si el nucleo es lineal o no, por defecto es verdadero
    """
    self.l=l
    self.h=h
    self.w1=w1
    self.w2=w2
    self.w3=w3
    self.w4=w4
    self.d=d
    self.perm=mu_r*mu_0
    self.vueltas=N
    self.lineal=lineal
    self.hay_reluctancia=False                #AUN NO SE HA HALLADO LA RELUCTANCIA        
 
  def calcular_reluctancia(self):
    self.reluctancia=1/(self.perm*self.d)*((self.h-(self.w1-self.w2)/2)*(1/self.w3+1/self.w4)+(self.l-(self.w4+self.w3)/2)*(1/self.w1+1/self.w2))
    self.hay_reluctancia=True
    return(self.reluctancia)
  def calcular_flujo(self,corriente):
    if (self.hay_reluctancia):      
      self.flujo=corriente*self.vueltas/self.reluctancia
    else:
      self.calcular_reluctancia()
      self.flujo=corriente*self.vueltas/self.reluctancia
    return self.flujo

#******************////////////*********************////////////////***********

class NucleoCompuesto:        
  def __init__(self,l1,l2,h,w1,w2,w3,w4,w5,d,mu_r ,N, lineal=True):    
    self.l1=l1
    self.l2=l2
    self.h=h
    self.w1=w1
    self.w2=w2
    self.w3=w3
    self.w4=w4
    self.w5=w4
    self.d=d
    self.perm=mu_r*mu_0
    self.vueltas=N
    self.lineal=lineal
    self.hay_reluctancia=False                #AUN NO SE HA HALLADO LA RELUCTANCIA        

  def calcular_reluctancia(self):
    self.rt1=(1/(self.perm*self.d))*((self.l1-self.w3/2)*(1/self.w2+1/self.w1)+(2*self.h-(self.w2+self.w1)/(2*self.w3)))
    self.rt2=(1/(self.perm*self.d))*((self.l2-self.w5/2)*(1/self.w2+1/self.w1)+(2*self.h-(self.w2+self.w1)/(2*self.w5)))
    self.r1=(2*self.h-(self.w1+self.w2))/(self.perm*self.w4*self.d)
    
    self.reluctancia=paralelo(self.rt1,self.rt2)+self.r1         #RELUCTANCIA TOTAL
    self.hay_reluctancia=True                 #ATRIBUTO INDICATIVO DE RELUCTANCIA
    return (self.reluctancia)
  def calcular_flujo(self,corriente):
    if (not self.hay_reluctancia):      
      self.calcular_reluctancia()       
    self.flujo=np.zeros(3)
    F=corriente*self.vueltas
    self.flujo[0]=F/self.reluctancia                               #FLUJO TOTAL
    self.flujo[1]=self.flujo[0] *  self.rt2/(self.rt2+self.rt1)     #FLUJO DE LA PRIMERA MALLA MAGNÉTICA
    self.flujo[2]=self.flujo[0] *  self.rt1/(self.rt2+self.rt1)   #FLUJO DE LA SEGUNDA MALLA MAGNÉTICA
    return self.flujo

In [ ]:
#CASO: NÚCLEO SIMPLE

@interact(vueltas=(1,1000,10),permeabilidad_relativa=(1,30,1), largo=(0.1,4,0.01), area=(0.001,0.1,0.001),corriente=(0,30,0.1), lineal=True,a=(0.4,1.2,0.01))

def mostrar_nucleo(vueltas=200, permeabilidad_relativa=10, largo=1.5, area=0.05,corriente=15, lineal=True, a=0.85):
  F=NucleoSimple(area,largo,permeabilidad_relativa,vueltas,a=a)  
  F.lineal=lineal
  print("Reluctancia total:", F.calcular_reluctancia(), " [A vuelta /weber]")
  print("Flujo magnético ideal: ", F.calcular_flujo(corriente), " [Weber]")
  F.plot()  

interactive(children=(IntSlider(value=200, description='vueltas', max=1000, min=1, step=10), IntSlider(value=1…

In [ ]:
#CASO: NÚCLEO DE UNA SOLA MALLA

@interact(vueltas=(1,1000,1),permeabilidad_relativa=(1,30,1), l=(0.6,2,0.01),h=(0.01,2,0.01),w1=(0.01,0.3,0.01),w2=(0.01,0.3,0.01),
          w3=(0.01,0.3,0.01),w4=(0.01,0.3,0.01),d=(0.01,0.3,0.01) ,corriente=(0,30,0.1))


def mostrar_nucleo(vueltas=200, permeabilidad_relativa=10, l=1.5,h=1.5, w1=0.1,w2=0.1,w3=0.1,w4=0.1,d=0.1,corriente=15):
  F=Nucleo(l,h,w1,w2,w3,w4,d,permeabilidad_relativa,vueltas)
  print("Reluctancia:", F.calcular_reluctancia(), " [A vuelta /weber]")
  print("Flujo magnético: ", F.calcular_flujo(corriente), " [Weber]")

interactive(children=(IntSlider(value=200, description='vueltas', max=1000, min=1), IntSlider(value=10, descri…

In [ ]:
#CASO: NÚCLEO DE DOS "MALLAS" MAGNÉTICAS

@interact(vueltas=(1,1000,1),permeabilidad_relativa=(1,30,1), l1=(0.01,1,0.01),l2=(0.01,1,0.01),w1=(0.01,0.3,0.01),w2=(0.01,0.3,0.01),
          w3=(0.01,0.3,0.01),w4=(0.01,0.3,0.01),w5=(0.01,0.3,0.01),d=(0.01,0.3,0.01) ,corriente=(0,30,0.1))


def mostrar_nucleo(vueltas=200, permeabilidad_relativa=10, l1=1.5,l2=1.5,h=1.5, w1=0.1,w2=0.1,w3=0.1,w4=0.1,w5=0.1,d=0.1,corriente=15):
  F=NucleoCompuesto(l1,l2,h,w1,w2,w3,w4,w5,d,permeabilidad_relativa,vueltas)
  print("Reluctancia:", F.calcular_reluctancia(), " [A vuelta /weber]")
  print("Flujo magnético: ", F.calcular_flujo(corriente), " [Weber]")

interactive(children=(IntSlider(value=200, description='vueltas', max=1000, min=1), IntSlider(value=10, descri…